In [1]:
#lets import the libraries we will use
from scikits.crab import datasets
from sklearn.datasets.base import Bunch
import numpy as np

In [2]:
import sqlalchemy
import pyodbc

#to hold the data from our database
movies_data = []
try:
 
    import urllib
    params = urllib.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=.\MSSQLSERVER_ENT;DATABASE=Antrak;Trusted_Connection=yes;")
 
    engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s"' % params)
    
    #insert customer and country searched into database
    engine.execute("insert into antrak.dbo.recommender_customers_tb values ('John','UK'), ('John','UK'),('John','UK'), ('Mike','US')")
    
    result = engine.execute("select * from antrak.dbo.recommender_rankings")
    
    for row in result:
        movies_data.append(row[0:3])
        
except (RuntimeError, TypeError, NameError):
    print('Error in Conneccting')
    print(RuntimeError, TypeError, NameError)


The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [3]:
#print(movies_data)

In [4]:
#lets check the data type of the returned data from the database
type(movies_data)

list

In [5]:
#lets convert the list of tuples to array as the recommender needs
data_m = np.asarray(movies_data)

In [6]:
#lets check the array data and format
data_m

array([['Freya', 'LB', '2.6'],
       ['Freya', 'LB', '2.5'],
       ['Amelia', 'SC', '2.6'],
       ..., 
       ['John', 'UK', '2.0'],
       ['Elvis', 'UK', '3.0'],
       ['John', 'US', '4.0']], 
      dtype='|S11')

In [7]:
#let define a function to load the data from the array

import pandas as pd
def load_sample_data():

    item_ids = []
    user_ids = []
    data_songs = {}
    for user_id, item_id, rating in data_m:
        if user_id not in user_ids:
            user_ids.append(user_id)
        if item_id not in item_ids:
            item_ids.append(item_id)
        u_ix = user_ids.index(user_id) + 1
        i_ix = item_ids.index(item_id) + 1
        data_songs.setdefault(u_ix, {})
        data_songs[u_ix][i_ix] = float(rating)

    data_t = []
    for no, item_id in enumerate(item_ids):
        data_t.append((no + 1, item_id))
    data_titles = dict(data_t)

    data_u = []
    for no, user_id in enumerate(user_ids):
        data_u.append((no + 1, user_id))
    data_users = dict(data_u)

    return Bunch(data=data_songs, item_ids=data_titles,
                 #user_ids=data_users, DESCR=fdescr.read())
                 user_ids=data_users)

In [8]:
#let's call the function and view sample data
movies = load_sample_data()

In [9]:
#songs = datasets.load_sample_songs()

In [61]:
#lets view some sample data
import pprint ## to make printed items clearer
#pprint.pprint(movies.data)

In [60]:
#pprint.pprint(movies.user_ids)

In [12]:
movies.user_ids.get(148)

'Finley'

In [59]:
#pprint.pprint(movies.item_ids)

In [14]:
from scikits.crab.models import MatrixPreferenceDataModel

In [15]:
#Build the model
model = MatrixPreferenceDataModel(movies.data)

In [16]:
from scikits.crab.metrics import pearson_correlation

In [17]:
from scikits.crab.similarities import UserSimilarity

In [18]:
#Build the similarity
similarity = UserSimilarity(model, pearson_correlation)

In [19]:
from sklearn.base import BaseEstimator


In [20]:
from scikits.crab.recommenders.knn import UserBasedRecommender

In [21]:
#build the user Based recommender
recommender = UserBasedRecommender(model, similarity, with_preference=True)

In [22]:
#recommend item for a user 
rec_items = recommender.recommend(12)

C:\Users\adaba\Anaconda3\envs\gl-env\lib\site-packages\scikits\crab\recommenders\knn\classes.py:538: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 20 but corresponding boolean dimension is 6
  similarities = similarities[~np.isnan(prefs)]
C:\Users\adaba\Anaconda3\envs\gl-env\lib\site-packages\scikits\crab\recommenders\knn\classes.py:538: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 20 but corresponding boolean dimension is 5
  similarities = similarities[~np.isnan(prefs)]
C:\Users\adaba\Anaconda3\envs\gl-env\lib\site-packages\scikits\crab\recommenders\knn\classes.py:538: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 20 but corresponding boolean dimension is 7
  similarities = similarities[~np.isnan(prefs)]
C:\Users\adaba\Anaconda3\envs\gl-env\lib\site-packages\scikits\crab\recommenders\knn\classes.py:538: VisibleDeprecatio

In [23]:
# get the type of data returned by the recommender
type(rec_items)

list

In [24]:
#how many were recmmended for him
len(rec_items)

67

In [25]:
#get all recommended destinations
#rec_items

[(6, 4.4714285714285724),
 (166, 4.0499999999999998),
 (217, 3.8250000000000002),
 (160, 2.4000000000000004),
 (114, 2.4000000000000004),
 (22, 2.3400000000000003),
 (175, 2.3200000000000003),
 (147, 2.3200000000000003),
 (91, 2.3200000000000003),
 (5, 2.3200000000000003),
 (92, 2.2285714285714286),
 (174, 2.2125000000000004),
 (99, 2.2125000000000004),
 (159, 2.166666666666667),
 (126, 2.166666666666667),
 (3, 2.166666666666667),
 (128, 2.1333333333333333),
 (109, 2.0333333333333332),
 (216, 2.0),
 (213, 2.0),
 (210, 2.0),
 (202, 2.0),
 (198, 2.0),
 (196, 2.0),
 (186, 2.0),
 (182, 2.0),
 (181, 2.0),
 (168, 2.0),
 (167, 2.0),
 (164, 2.0),
 (152, 2.0),
 (150, 2.0),
 (148, 2.0),
 (131, 2.0),
 (115, 2.0),
 (112, 2.0),
 (110, 2.0),
 (108, 2.0),
 (86, 2.0),
 (84, 2.0),
 (71, 2.0),
 (62, 2.0),
 (41, 2.0),
 (38, 2.0),
 (28, 2.0),
 (119, 1.9624999999999999),
 (117, 1.9624999999999999),
 (111, 1.9624999999999999),
 (89, 1.9624999999999999),
 (75, 1.9624999999999999),
 (67, 1.9624999999999999),


In [26]:
#get top 5 recommended destinations
top5 = rec_items[:5]
top5

[(6, 4.4714285714285724),
 (166, 4.0499999999999998),
 (217, 3.8250000000000002),
 (160, 2.4000000000000004),
 (114, 2.4000000000000004)]

In [27]:
#rec_items[:]

In [28]:
#get a destination names
movies.item_ids.get(149)

'PG'

In [29]:
#let's check the type of data the item ids are in
type(movies.item_ids)

dict

In [30]:
#get the ids of all recommmended countries
rec_count_id = [item[0] for item in top5]
rec_count_id

[6, 166, 217, 160, 114]

In [31]:
#lets get the names of the top 5 recommended countries
rec_countries = [movies.item_ids[x] for x in rec_count_id]

In [32]:
#top five recommended countries
rec_countries

['VI', 'TH', 'CR', 'US', 'UY']

In [33]:
# get top N recommended countries
# @user_id : id of the customer to be recommended a product
# @n: Number of recommendations to make , with the highest rated recommended products in top

def topNRecs(user_id, n):
    
    #lets ingore the warnings
    import warnings
    warnings.filterwarnings('ignore')
    
    rec_items = recommender.recommend(user_id)
    
    #get top n ids
    topN = rec_items[:n]
    
    #get the ids of all recommmended countries
    rec_count_id = [item[0] for item in topN]
    
    #lets get the names of the top 5 recommended countries
    rec_countries = [movies.item_ids[x] for x in rec_count_id]
    
    return rec_countries
    
    
    

In [34]:
# get top N recommended countries along with the name of the user
# @user_id : id of the customer to be recommended a product
# @n: Number of recommendations to make , with the highest rated recommended products in top

def topNRecsUser(user_id, n):
    
    #lets ingore the warnings
    import warnings
    warnings.filterwarnings('ignore')
    
    recdit = {}
    rec_items = recommender.recommend(user_id)
    
    #get top n ids
    topN = rec_items[:n]
    
    #get the ids of all recommmended countries
    rec_count_id = [item[0] for item in topN]
    
    #lets get the names of the top 5 recommended countries
    rec_countries = [movies.item_ids[x] for x in rec_count_id]
    recdit['customer'] = movies.user_ids.get(user_id)
    recdit['recommended countries'] = rec_countries
    
    #recdit[movies.user_ids.get(user_id)] = rec_countries
    
    return recdit
    
    
    

In [35]:
#lets recommend top 10 destinations for a user
topNRecs(8, 10)

['VC', 'CR', 'NZ', 'CD', 'VI', 'BD', 'BY', 'BB', 'BZ', 'BE']

In [36]:
#lets get the total number of recommendations made for a customer
def totalRecs(user_id):
    import warnings
    warnings.filterwarnings('ignore')
    
    rec_items = recommender.recommend(user_id)
    return len(rec_items)
    

In [37]:
#check total number of recommendations for a specified user id
totalRecs(102)

147

In [38]:
# get all recommended countries
# @user_id : id of the customer to be recommended a product
# @n: Number of recommendations to make , with the highest rated recommended products in top

def allRecs(user_id):
    
    #lets ingore the warnings
    import warnings
    warnings.filterwarnings('ignore')
    
    rec_items = recommender.recommend(user_id)
    
    #get top n ids
    allrec = rec_items[:]
    
    #get the ids of all recommmended countries
    rec_count_id = [item[0] for item in allrec]
    
    #lets get the names of the top 5 recommended countries
    rec_countries = [movies.item_ids[x] for x in rec_count_id]
    
    return rec_countries

In [39]:
# check all recommended countries for a specified user id
#allRecs(8)

In [40]:
#get the recommender user id of a particular customer by his name
print(list(movies.user_ids.keys())[list(movies.user_ids.values()).index('Nana')])

228


In [41]:
#lets the user id of  a customer in the recommender system by using his name
userid = list(movies.user_ids.keys())[list(movies.user_ids.values()).index('Lewis')]
userid

160

In [42]:
#lets use that userid to get the top 5 recommendations for the customer
result = topNRecs(userid, 5)
result

['VC', 'KG', 'CD', 'PE', 'PL']

In [43]:
#lets the name of a customer and the top 5 countries recommended to him
userRec = topNRecsUser(102,5)

In [44]:
userRec

{'customer': 'Jade', 'recommended countries': ['VC', 'CR', 'CD', 'VI', 'BD']}

In [45]:
#lets save the customer and the recommended countries to a json file
import json
with open('data.json', 'w') as outfile:
    json.dump(userRec, outfile, indent = 4)

In [51]:
# get top N recommended countries along with the name of the user
# @user_id : id of the customer to be recommended a product
# @n: Number of recommendations to make , with the highest rated recommended products in top

def topNRecsUserName(user_name, n):
    try:
        int(user_name)
        print("Please enter a valid String")
    except ValueError:
        print("Thanks for supplying name")
    #user_id = list(movies.user_ids.keys())[list(movies.user_ids.values()).index("'"+ user_name +"'")]
    user_id = list(movies.user_ids.keys())[list(movies.user_ids.values()).index(user_name)]
    
    #lets ingore the warnings
    import warnings
    warnings.filterwarnings('ignore')
    
    recdit = {}
    rec_items = recommender.recommend(user_id)
    
    #get top n ids
    topN = rec_items[:n]
    
    #get the ids of all recommmended countries
    rec_count_id = [item[0] for item in topN]
    
    #lets get the names of the top 5 recommended countries
    rec_countries = [movies.item_ids[x] for x in rec_count_id]
    recdit['customer'] = movies.user_ids.get(user_id)
    recdit['recommended countries'] = rec_countries
    
    #recdit[movies.user_ids.get(user_id)] = rec_countries
    
    return recdit
    
    
    

In [58]:
topNRecsUserName('John', 5)

Thanks for supplying name


{'customer': 'John', 'recommended countries': ['TH', 'VI', 'CR', 'CD', 'NZ']}